In [4]:
import json

In [38]:
from glob import glob
class Artifact(object):
    
    def __init__(self, expname, ccd, problem, x, y):
        self.expname = expname
        self.ccd = ccd
        self.problem = problem
        self.x = x
        self.y = y
        

def load_release_artifacts(artifact_base):
    files = glob(artifact_base+'*')
    artifacts = []
    for f in files:
        with open(f, 'r') as fp:
            arts = json.load(fp)
            for art in arts:
                if ~art['false_positive']:
                    oart = Artifact(art['expname'], art['ccd'],\
                                    art['problem'], art['x'], \
                                    art['y'])
                    artifacts.append(oart)

    artifacts.sort(key=lambda x : x.expname)
    artifacts = np.array(artifacts)
    
    return artifacts

def create_design_matrix(image_glob, artifacts, xdiam=100, ydiam=100):
    
    assert((xdiam<2048) & (ydiam<4096))
    
    edirs  = np.array(glob(image_glob))
    edirs = np.sort(edirs)
    features = []
    labels = []
    loaded = []
    imgcache = None
    
    for art in artifacts:
        xlflag = xrflag = ylflag = yrflag = False
        if art.expname in loaded:
            if art.x-xdiam/2<0:
                minx = 0
            else:
                minx = art.x-xdiam/2
            if art.x+xdiam/2>imgcache.shape[0]:
                maxx = imgcache.shape[0]
            else:
                maxx = art.x+xdiam/2

            if art.y-ydiam/2<0:
                miny = 0
            else:
                miny = art.y-ydiam/2
            if art.y+ydiam/2>imgcache.shape[1]:
                maxy = imgcache.shape[1]
            else:
                maxy = art.y+ydiam/2              

            stamp = imgcache[minx:maxx, miny:maxy]
            features.append(stamp.flatten())
            labels.append(art.problem)
            
    return np.array(features), np.array(labels)

def handle_edges(endpts, image, x, y, xdiam=100, ydiam=100):
    
    stamp = np.zeros((xdiam,ydiam))
    
    stamp[max(0,xdiam-(endpts[0][1]-endpts[0][1])):
          min(xdiam, endpts[0])
    


In [39]:
artifacts = load_release_artifacts('/nfs/slac/g/ki/ki23/des/jderose/des/se_exposures/metadata/SVA1/')

In [48]:
ig = '/nfs/slac/g/ki/ki23/des/jderose/des/se_exposures/OPS/red/20130*/red*/*'


['/nfs/slac/g/ki/ki23/des/jderose/des/se_exposures/OPS/red/20130715185024_20121221/red/DECam_00163238', '/nfs/slac/g/ki/ki23/des/jderose/des/se_exposures/OPS/red/20130716074552_20121222/red/DECam_00163490', '/nfs/slac/g/ki/ki23/des/jderose/des/se_exposures/OPS/red/20130712064117_20121207/red/DECam_00159178', '/nfs/slac/g/ki/ki23/des/jderose/des/se_exposures/OPS/red/20130805180050_20121101/red/DECam_00145351']
